### NAT + NVIDIA RAG MCP end-to-end notebook

This notebook demonstrates how to:
1. Start the NVIDIA RAG MCP server in `streamable_http` mode.
2. Create a collection and upload a sample document via the MCP client.
3. Configure a NAT workflow using `nvidia_rag_mcp.yaml`.
4. Start a NAT server that uses the RAG MCP server as a tool.
5. Call the NAT workflow with a natural language query.
6. Clean up the collection and stop the NAT and MCP servers.

**Prerequisites**

- End-to-end NVIDIA RAG workflow is up and running (per the [RAG quickstart](https://github.com/NVIDIA-AI-Blueprints/rag/blob/develop/docs/deploy-docker-self-hosted.md)).
- `nat` CLI is installed and on your `PATH` (to install it, see the [NeMo Agent Toolkit installation guide](https://docs.nvidia.com/nemo/agent-toolkit/latest/quick-start/installing.html)).
- MCP server/client dependencies are installed.

In [ ]:
# Optional: ensure MCP dependencies are installed in this environment.
# You can skip this cell if you've already installed them.
# Uncomment and run the next line if needed:
# %pip install -r ../nvidia_rag_mcp/requirements.txt

In [ ]:
"""Start NVIDIA RAG MCP server in streamable_http mode on port 9902.

This cell launches the MCP server as a background subprocess and
registers a cleanup hook to terminate it when the kernel shuts down.
"""

import os
import sys
import atexit
import time
import subprocess

MCP_PORT = 9902
MCP_HOST = "127.0.0.1"

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
server_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_server.py")

cmd = [
    sys.executable,
    server_path,
    "--transport",
    "streamable_http",
    "--host",
    MCP_HOST,
    "--port",
    str(MCP_PORT),
]

print("Launching MCP server:", " ".join(cmd))

mcp_proc = subprocess.Popen(cmd)
atexit.register(lambda: mcp_proc and mcp_proc.poll() is None and mcp_proc.terminate())

time.sleep(2.0)
print(f"MCP server PID: {mcp_proc.pid} (http://{MCP_HOST}:{MCP_PORT}/mcp)")

In [ ]:
"""Create a collection and upload a sample document via the MCP client.

This cell uses the MCP client in streamable_http mode pointed at the
MCP server started above on port 9902.
"""

import json
import subprocess
import os
import sys

STREAMABLE_HTTP_URL = "http://127.0.0.1:9902/mcp"
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
client_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_client.py")

COLLECTION = "my_collection"
pdf_path = os.path.join(repo_root, "data", "multimodal", "product_catalog.pdf")

print("=" * 80)
print("Creating collection via MCP...")
print("=" * 80)
create_args = json.dumps({"collection_names": [COLLECTION]})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=streamable_http",
    f"--url={STREAMABLE_HTTP_URL}",
    "--tool=create_collections",
    f"--json-args={create_args}",
], check=False)

print("\n" + "=" * 80)
print("Uploading document via MCP...")
print("=" * 80)
upload_args = json.dumps({
    "collection_name": COLLECTION,
    "file_paths": [pdf_path],
    "blocking": True,
    "generate_summary": True,
    "split_options": {"chunk_size": 512, "chunk_overlap": 150},
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=streamable_http",
    f"--url={STREAMABLE_HTTP_URL}",
    "--tool=upload_documents",
    f"--json-args={upload_args}",
], check=False)

print("\nDone setting up collection and document for NAT + MCP demo.")

### Save `nvidia_rag_mcp.yaml` for NAT

Create a file named `nvidia_rag_mcp.yaml` in your project root (for example, in the same directory where you will run `nat serve`) and paste the following content, then save the file.

At a high level this YAML:
- **Defines an MCP client** (`function_groups.nvidia_rag_mcp`) that connects to the NVIDIA RAG MCP server over `streamable-http` at `http://localhost:9902/mcp` and exposes the `generate` tool to NAT.
- **Configures the LLM** (`llms.nim_llm`) that the agent uses when deciding how to call the MCP tool.
- **Builds a ReAct agent workflow** (`workflow`) that wires the `nvidia_rag_mcp` tool group and `nim_llm` model together.

You normally only need to change this file if you:
- Point the MCP server to a different host/port (update the `url`), or
- Want to use a different LLM (update `model_name` and related settings).

```yaml

function_groups:
  nvidia_rag_mcp:
    _type: mcp_client
    server:
      transport: streamable-http
      url: "http://localhost:9902/mcp"
    include:
      - generate

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0
    max_tokens: 1024

workflow:
  _type: react_agent
  tool_names:
    - nvidia_rag_mcp
  llm_name: nim_llm
  verbose: true
  retry_parsing_errors: true
  max_retries: 3
```

### Start the NAT server

In a separate terminal (from the project root where `nvidia_rag_mcp.yaml` is saved), start the NAT server pointing at the MCP config:

```bash
nat serve --config_file nvidia_rag_mcp.yaml --port 8000
```

Leave this process running while you execute the next cell to send a query via `nat run`.

### Run `nat run`

Open a separate terminal **in the directory where `nvidia_rag_mcp.yaml` is saved** and run:

```bash
nat run --config_file nvidia_rag_mcp.yaml --input "My rag question is: Tell me about Ratan Basket Shoulder Bag. Use collection my_collection"
```

This will invoke the workflow defined in `nvidia_rag_mcp.yaml`, which uses the NVIDIA RAG MCP server (running on `http://localhost:9902/mcp`) to answer based on the document you uploaded.

In [ ]:
"""Delete the demo collection via the MCP client.

Run this cell after you are done experimenting with the RAG collection.
"""

import json
import subprocess
import os
import sys

STREAMABLE_HTTP_URL = "http://127.0.0.1:9902/mcp"
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
client_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_client.py")

COLLECTION = "my_collection"

print("Deleting collection via MCP...\n")
delete_args = json.dumps({"collection_names": [COLLECTION]})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=streamable_http",
    f"--url={STREAMABLE_HTTP_URL}",
    "--tool=delete_collections",
    f"--json-args={delete_args}",
], check=False)

### Stop the NAT server

To stop the NAT server you started with:

```bash
nat serve --config_file nvidia_rag_mcp.yaml --port 8000
```

Go to that terminal and press `Ctrl+C` to terminate it.

Alternatively, from a shell in the same machine, you can locate and kill the process explicitly, for example:

```bash
ps aux | grep "nat serve --config_file nvidia_rag_mcp.yaml --port 8000"
# Note the PID from the output, then run
kill <PID>
``` 

In [ ]:
"""Stop the MCP server started earlier in this notebook.

This cell uses the `mcp_proc` handle created in the server-start cell.
If the process is still running, it will be terminated.
"""

try:
    mcp_proc
except NameError:
    print("No MCP server process handle (`mcp_proc`) found in this session.")
else:
    if mcp_proc is not None and mcp_proc.poll() is None:
        print(f"Terminating MCP server PID {mcp_proc.pid}...")
        mcp_proc.terminate()
        try:
            mcp_proc.wait(timeout=10)
        except Exception:
            print("MCP server did not exit in time; sending kill()...")
            mcp_proc.kill()
        print("MCP server stopped.")
    else:
        print("MCP server process is not running.")

### Troubleshooting

- **MCP server fails to start or hangs**
  - Check that NVIDIA RAG and Ingestor services are up and healthy (see the [RAG quickstart](https://github.com/NVIDIA-AI-Blueprints/rag/blob/develop/docs/deploy-docker-self-hosted.md)).
  - Ensure port `9902` is free: `fuser -k 9902/tcp` (Linux) and rerun the server cell.

- **MCP client calls (create/upload/delete) fail**
  - Verify the MCP server cell printed a valid PID and URL `http://127.0.0.1:9902/mcp`.
  - Run the MCP client commands from the notebook again and inspect any error text in the cell output.

- **`nat serve` cannot connect to MCP**
  - Confirm the MCP server cell is still running (no error in its output, and the stop-MCP cell reports it as running).
  - Make sure the `url` in `nvidia_rag_mcp.yaml` matches the MCP server address (default: `http://localhost:9902/mcp`).

- **`nat run` errors or returns empty/irrelevant answers**
  - Confirm the collection (`my_collection`) exists and the upload cell completed successfully.
  - Adjust the `--input` prompt to clearly mention the collection name and the document you uploaded.

- **Environment or dependency issues**
  - Ensure `nat` is installed and on your `PATH` (see the [NeMo Agent Toolkit installation guide](https://docs.nvidia.com/nemo/agent-toolkit/latest/quick-start/installing.html)).
  - (Optional) Rerun the first pip cell to install MCP dependencies in the current environment.